
 # WatsonX Basic Multi-agent Collaboration

 A single agent can usually operate effectively using a handful of tools within a single domain, but even using powerful models like `gpt-4`, it can be less effective at using many tools.

 One way to approach complicated tasks is through a "divide-and-conquer" approach: create an specialized agent for each task or domain and route tasks to the correct "expert".

 This notebook (inspired by the paper [AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155), by Wu, et. al.) shows one way to do this using LangGraph.



The resulting graph will look something like the following diagram:

![multi_agent diagram](./img/simple_multi_agent_diagram.png)

Before we get started, a quick note: this and other multi-agent notebooks are designed to show _how_ you can implement certain design patterns in LangGraph. If the pattern suits your needs, we recommend combining it with some of the other fundamental patterns described elsewhere in the docs for best performance.

In [1]:
import logging
from typing import Any, Dict, List, Optional, Sequence, Type, Union, Callable, Literal
from pydantic import BaseModel, Field
from langchain_core.tools import BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import Runnable
from langchain_ibm import WatsonxLLM as BaseWatsonxLLM
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.outputs import LLMResult, Generation, GenerationChunk
from langchain_core.messages import BaseMessage
from langchain_core.language_models import LanguageModelInput
from dotenv import load_dotenv
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
import os
import getpass

logger = logging.getLogger(__name__)

# Simplified loading of environment variables and IBM connection parameters
load_dotenv()

def set_env(var: str):
    env_var = os.getenv(var)
    if not env_var:
        env_var = getpass.getpass(f"{var}: ")
        os.environ[var] = env_var
    return env_var

class IbmConnectionParams(BaseModel):
    api_key: str
    project_id: str
    url: str
    credentials: dict[str, str]

    def __init__(self, api_key: str, project_id: str, url: str) -> None:
        super().__init__(api_key=api_key, project_id=project_id, url=url, credentials={"url": url, "apikey": api_key})

# Set IBM connection parameters
ibm_params = IbmConnectionParams(
    api_key=set_env("WATSONX_API_KEY"),
    project_id=set_env("PROJECT_ID"),
    url = set_env("WATSONX_URL")
)

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.TEMPERATURE: 0.7,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

class WatsonxLLM(BaseWatsonxLLM):
    """Extended IBM watsonx.ai large language models."""
    # Define the default parameters as class variables
    DEFAULT_MODEL_ID = "ibm/granite-13b-instruct-v2"
    DEFAULT_URL = ibm_params.url
    DEFAULT_APIKEY = ibm_params.api_key
    DEFAULT_PROJECT_ID = ibm_params.project_id
    DEFAULT_PARAMS = parameters    
    bound_tools: Optional[List[BaseTool]] = Field(default=None, exclude=True)

    def __init__(self, *args, tools: Optional[List[BaseTool]] = None, model_id: Optional[str] = None, **kwargs):
        if not model_id and 'deployment_id' not in kwargs:
            raise ValueError("One of 'model_id' or 'deployment_id' parameters should be set.")
        super().__init__(model_id=model_id, *args, **kwargs)
        self.bound_tools = tools or []

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> LLMResult:
        params = self._get_chat_params(stop=stop)
        should_stream = stream if stream is not None else self.streaming
        if should_stream:
            if len(prompts) > 1:
                raise ValueError(f"WatsonxLLM currently only supports single prompt, got {prompts}")
            generation = GenerationChunk(text="")
            stream_iter = self._stream(prompts[0], stop=stop, run_manager=run_manager, **kwargs)
            for chunk in stream_iter:
                if generation is None:
                    generation = chunk
                else:
                    generation += chunk
            assert generation is not None
            if isinstance(generation.generation_info, dict):
                llm_output = generation.generation_info.pop("llm_output")
                return LLMResult(generations=[[generation]], llm_output=llm_output)
            return LLMResult(generations=[[generation]])
        else:
            print("Step 1 - Starting Tool Evaluation")
                        
            if self.bound_tools:
                print("Step 1.1 self.bound_tools")
                print("type(self.bound_tools)", type(self.bound_tools)) 
                print("self.bound_tools", self.bound_tools)
                
                tool_output = self._evaluate_tools(self.bound_tools, prompts[0])
                logger.info("Tool output: %s", tool_output)

                system_prompt = (
                    f"You are an assistant with access to web search results. "
                    f"Provide a detailed answer to the user's query.\n\n"
                    f"User Query: {prompts[0]}\n\n"
                    f"Using the information below:\n"
                    f"Web Search Results: {tool_output}\n\n"
                )
                prompts[0] = system_prompt

            print("prompts", prompts)

            if not isinstance(params, dict):
                raise ValueError(f"Expected params to be a dictionary, got {type(params)}")

            response = self.watsonx_model.generate(prompt=prompts, params=params, **kwargs)
            return self._create_llm_result(response)


    def _evaluate_tools(self, tool_instances: List[BaseTool], input_text: str) -> str:
        combined_output = []
        print(f"Debug: Starting tool evaluation for input: {input_text}")
        print("tool_instances:", tool_instances)

        for tool in tool_instances:
            try:
                # Invoke the tool with the provided input
                result = tool.invoke(input_text)
                print(f"Debug: Result from tool {tool.name}: {result}")

                # Check the type of result and handle it accordingly
                if isinstance(result, str):
                    content = f"WebSearch Results: {result}"
                elif isinstance(result, list) and all(isinstance(item, dict) for item in result):
                    content = "WebSearch Results: " + " ".join(item.get('content', '') for item in result)
                elif isinstance(result, dict):
                    content = "WebSearch Results: " + result.get('content', 'Unknown response format')
                else:
                    content = "WebSearch Results: Invalid format received from tool"

                # Append the processed content to the combined output
                combined_output.append(content)

            except Exception as e:
                error_message = f"Error invoking tool {tool.name}: {str(e)}"
                print(f"Debug: {error_message}")
                combined_output.append(error_message)

        final_output = "\n\n".join(combined_output)
        print(f"Debug: Final combined output: {final_output}")
        return final_output

    @classmethod
    def bind_tools(
        cls,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        model_id: Optional[str] = None,
        url: Optional[str] = None,
        apikey: Optional[str] = None,
        project_id: Optional[str] = None,
        params: Optional[Dict[str, Any]] = None,
        tool_choice: Optional[Union[Dict[str, str], Literal["any", "auto"], str]] = None,
        **kwargs: Any,
    ) -> 'WatsonxLLM':
        formatted_tools = [convert_to_openai_tool(tool)["function"] for tool in tools]
        # Initialize WatsonxLLM with the provided parameters
        instance = cls(
            model_id=model_id,
            url=url,
            apikey=apikey,
            project_id=project_id,
            params=params,
            **kwargs
        )
        instance.bound_tools = tools
        if tool_choice is not None:
            kwargs["tool_choice"] = tool_choice
        return instance

    def _create_llm_result(self, response: List[dict]) -> LLMResult:
        generations = []
        for res in response:
            results = res.get("results")
            if results:
                finish_reason = results[0].get("stop_reason")
                gen = Generation(
                    text=results[0].get("generated_text"),
                    generation_info={"finish_reason": finish_reason},
                )
                generations.append([gen])
        final_token_usage = self._extract_token_usage(response)
        llm_output = {
            "token_usage": final_token_usage,
            "model_id": self.model_id,
            "deployment_id": self.deployment_id,
        }
        return LLMResult(generations=generations, llm_output=llm_output)

# Example usage:
tool = TavilySearchResults(max_results=4)

# Bind the tool and pass all necessary parameters inside `bind_tools`
llm_with_tools = WatsonxLLM.bind_tools(
    tools=[tool],
    model_id="ibm/granite-13b-instruct-v2",
    url=ibm_params.url,
    apikey=ibm_params.api_key,
    project_id=ibm_params.project_id,
    params=parameters
)
# Invoke the model
#response = llm_with_tools.invoke("Who is Ruslan Magana?")
#print(response)


In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, StateGraph
import functools
import re
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
import operator
from typing import Annotated, Sequence, TypedDict

# Define the object passed between nodes
class AgentState(TypedDict):
    messages: Annotated[Sequence[AIMessage], operator.add]
    sender: str

# Tool Initialization
tavily_tool = TavilySearchResults(max_results=4)
repl = PythonREPL()

# Define Python REPL Tool
@tool
def python_repl(code: Annotated[str, "The python code to execute to generate your chart."]):
    """Use this to execute python code. If you want to see the output of a value, you should print it out with `print(...)`. This is visible to the user."""
    try:
        exec(code)
        result = ""
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."

# Function to handle the researcher data extraction
def extract_research_data(result):
    """Extract relevant research data from TavilySearchResults results."""
    extracted_data = []
    for entry in result:
        if 'content' in entry:
            extracted_data.append(entry['content'])
    return extracted_data if extracted_data else "No relevant data found."

# Simplified agent creation function
def create_agent(llm, tools, system_message: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful AI assistant, collaborating with other assistants. "
             "Use the provided tools to answer the question. If you can't answer fully, another assistant will help. "
             "Prefix final responses with FINAL ANSWER.\nTools available: {tool_names}.\n{system_message}"),
            MessagesPlaceholder(variable_name="messages"),
        ]
    ).partial(system_message=system_message, tool_names=", ".join([tool.name for tool in tools]))

    return prompt | llm.bind_tools(
        tools=tools,
        model_id="ibm/granite-13b-instruct-v2",
        url=ibm_params.url,
        apikey=ibm_params.api_key,
        project_id=ibm_params.project_id,
        params=parameters
    )

# Extract Python code from the result
def filter_python_code(result):
    """
    Extract all valid Python code blocks from the AI-generated response.
    Looks for code blocks enclosed within triple backticks (```python ... ```).
    Returns a list of code blocks, or a message if none are found.
    """
    code_blocks = re.findall(r"```python\s*(.*?)```", str(result), re.DOTALL)
    if code_blocks:
        return [code.strip() for code in code_blocks]
    else:
        return "No Python code blocks found."

# Function to invoke agents and handle their results
def agent_node(state, agent, name):
    result = agent.invoke(state)
    print(f"Debug: Agent {name} invoked with state: {state}")
    print(f"Debug: Result from agent {name}: {result}")

    # If the agent is the researcher, extract research data
    if name == "Researcher":
        if isinstance(result, str) and "Error" in result:
            print("Debug: Error occurred in Research Agent. Retrying...")
        else:
            result = extract_research_data(result)
            print(f"Debug: Extracted Research Data: {result}")

    # Filter Python code if invoking the chart generator
    if name == "chart_generator" or name == "code_generator":
        result = filter_python_code(result)

    # Handle different result types
    if hasattr(result, 'dict'):
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    else:
        result = AIMessage(content=str(result), name=name)

    return {
        "messages": [result],
        "sender": name,
    }

# Define agents and their nodes
research_agent = create_agent(
    WatsonxLLM, [tavily_tool], system_message="Provide accurate data for the chart generator."
)
chart_agent = create_agent(
    WatsonxLLM, [python_repl], system_message="Any charts you display will be visible to the user."
)

# Python Code Generator Agent (new)
code_generator_agent = create_agent(
    WatsonxLLM, [], system_message="Generate Python code using matplotlib for drawing the chart based on the researcher's data."
)

research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")
chart_node = functools.partial(agent_node, agent=chart_agent, name="chart_generator")
code_generator_node = functools.partial(agent_node, agent=code_generator_agent, name="code_generator")

# Create tool node
tools = [tavily_tool, python_repl]
tool_node = ToolNode(tools)

# Define router for graph transitions
def router(state) -> str:
    last_message = state["messages"][-1]
    print(f"Debug: Last message in state: {last_message}")
    
    if last_message.tool_calls:
        print("Debug: Routing to call_tool")
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        print("Debug: Routing to __end__")
        return "__end__"
    print("Debug: Continuing to next node")
    return "continue"

# Define the graph and its nodes
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("chart_generator", chart_node)
workflow.add_node("code_generator", code_generator_node)
workflow.add_node("call_tool", tool_node)
workflow.add_conditional_edges("Researcher", router, {"continue": "code_generator", "call_tool": "call_tool", "__end__": END})
workflow.add_conditional_edges("code_generator", router, {"continue": "chart_generator", "call_tool": "call_tool", "__end__": END})
workflow.add_conditional_edges("chart_generator", router, {"continue": "Researcher", "call_tool": "call_tool", "__end__": END})
workflow.add_conditional_edges("call_tool", lambda x: x["sender"], {"Researcher": "Researcher", "chart_generator": "chart_generator", "code_generator": "code_generator"})

# Set the entry point of the workflow
workflow.set_entry_point("Researcher")

# Compile the workflow for execution
graph = workflow.compile()

# Execute the graph stream with a test message
events = graph.stream(
    {
        "messages": [
            HumanMessage(content="Fetch the bitcoin price over the past 5 days, then draw a line graph of it.")
        ]
    },
    {"recursion_limit": 50}
)

# Output the results
for s in events:
    print(s)
    print("----")


Step 1 - Starting Tool Evaluation
Step 1.1 self.bound_tools
type(self.bound_tools) <class 'list'>
self.bound_tools [TavilySearchResults(max_results=4)]
Debug: Starting tool evaluation for input: System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to answer the question. If you can't answer fully, another assistant will help. Prefix final responses with FINAL ANSWER.
Tools available: tavily_search_results_json.
Extract the data in a list such can can be used for the chart generator.
Human: Fetch the bitcoin price over the past 5 days, then draw a line graph of it.
tool_instances: [TavilySearchResults(max_results=4)]
Debug: Result from tool tavily_search_results_json: HTTPError('400 Client Error: Bad Request for url: https://api.tavily.com/search')
Debug: Final combined output: WebSearch Results: HTTPError('400 Client Error: Bad Request for url: https://api.tavily.com/search')
prompts ["You are an assistant with access to web search results

# Agent Testing

In [3]:
def test_agent(agent_name: str, agent, test_input: str):
    """
    Function to test individual agents in the workflow.

    :param agent_name: Name of the agent to be tested (e.g., 'Researcher', 'chart_generator').
    :param agent: The agent object created using the create_agent function.
    :param test_input: The input message to be passed to the agent for testing.
    """
    # Create a mock state with test input
    state = {
        "messages": [HumanMessage(content=test_input)],
        "sender": agent_name
    }

    # Invoke the agent node
    result = agent_node(state, agent, agent_name)

    # Output the result for debugging and checking
    print(f"Agent Name: {agent_name}")
    print(f"Test Input: {test_input}")
    print(f"Agent Output: {result['messages'][0].content}")
    print("\n" + "-" * 40 + "\n")

In [4]:
# Usage Example:
# Test the Researcher agent
test_agent('Researcher', research_agent, "What is the Bitcoin price over the past 5 days?")


Step 1 - Starting Tool Evaluation
Step 1.1 self.bound_tools
type(self.bound_tools) <class 'list'>
self.bound_tools [TavilySearchResults(max_results=4)]
Debug: Starting tool evaluation for input: System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to answer the question. If you can't answer fully, another assistant will help. Prefix final responses with FINAL ANSWER.
Tools available: tavily_search_results_json.
Provide accurate data for the chart generator.
Human: What is the Bitcoin price over the past 5 days?
tool_instances: [TavilySearchResults(max_results=4)]
Debug: Result from tool tavily_search_results_json: [{'url': 'https://www.coingecko.com/en/coins/bitcoin/historical_data', 'content': 'Get Bitcoin (BTC) USD historical prices. Coins: 14,755. Exchanges: 1,205. Market Cap: ... All Coins Compare Coins Converter Global Chart Exchanges Crypto Exchanges Decentralized Exchanges Derivatives NFT NFT ... 2024-09-05 $1,144,983,177,152 $37,55

In [5]:
# Test the code generator agent
test_agent('code_generator', code_generator_agent, "Generate Python code to plot the Bitcoin price.")


Step 1 - Starting Tool Evaluation
prompts ["System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to answer the question. If you can't answer fully, another assistant will help. Prefix final responses with FINAL ANSWER.\nTools available: .\nGenerate Python code using matplotlib for drawing the chart based on the researcher's data.\nHuman: Generate Python code to plot the Bitcoin price."]
Debug: Agent code_generator invoked with state: {'messages': [HumanMessage(content='Generate Python code to plot the Bitcoin price.')], 'sender': 'code_generator'}
Debug: Result from agent code_generator: Plot the Bitcoin price over time.Add a line for the moving average.Plot a line for the exponential moving average.Add a legend.Save the plot as a .png image.FINAL ANSWER: #Here is the Python code you requested.
Agent Name: code_generator
Test Input: Generate Python code to plot the Bitcoin price.
Agent Output: No Python code blocks found.

----------------

In [6]:
# Test the chart generator agent
test_agent('chart_generator', chart_agent, "Plot the Bitcoin price using the provided data.")


Step 1 - Starting Tool Evaluation
Step 1.1 self.bound_tools
type(self.bound_tools) <class 'list'>
self.bound_tools [StructuredTool(name='python_repl', description='Use this to execute python code. If you want to see the output of a value, you should print it out with `print(...)`. This is visible to the user.', args_schema=<class 'pydantic.v1.main.python_replSchema'>, func=<function python_repl at 0x7eff4c79af80>)]
Debug: Starting tool evaluation for input: System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to answer the question. If you can't answer fully, another assistant will help. Prefix final responses with FINAL ANSWER.
Tools available: python_repl.
Any charts you display will be visible to the user.
Human: Plot the Bitcoin price using the provided data.
tool_instances: [StructuredTool(name='python_repl', description='Use this to execute python code. If you want to see the output of a value, you should print it out with `print(...

In [10]:


# Updated agent_node for Researcher (for testing):
def agent_node(state, agent, name):
    result = agent.invoke(state)
    print(f"Debug: Agent {name} invoked with state: {state}")
    print(f"Debug: Result from agent {name}: {result}")

    # If the agent is the researcher, extract research data
    if name == "Researcher":
        if isinstance(result, str) and "Error" in result:
            print("Debug: Error occurred in Research Agent. Retrying...")
        else:
            print(f"Debug: Extracted Research Data: {result}")

    # Filter Python code if invoking the chart generator
    if name == "chart_generator" or name == "code_generator":
        result = filter_python_code(result)

    # Handle different result types
    if hasattr(result, 'dict'):
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    else:
        result = AIMessage(content=str(result), name=name)

    return {
        "messages": [result],
        "sender": name,
    }

# Usage example:
# Test the Researcher agent again with the updated extraction logic
test_agent('Researcher', research_agent, "What is the Bitcoin price over the past 5 days?")


Step 1 - Starting Tool Evaluation
Step 1.1 self.bound_tools
type(self.bound_tools) <class 'list'>
self.bound_tools [TavilySearchResults(max_results=4)]
Debug: Starting tool evaluation for input: System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to answer the question. If you can't answer fully, another assistant will help. Prefix final responses with FINAL ANSWER.
Tools available: tavily_search_results_json.
Provide accurate data for the chart generator.
Human: What is the Bitcoin price over the past 5 days?
tool_instances: [TavilySearchResults(max_results=4)]
Debug: Result from tool tavily_search_results_json: [{'url': 'https://www.coingecko.com/en/coins/bitcoin/historical_data', 'content': 'Get Bitcoin (BTC) USD historical prices. Get Bitcoin (BTC) USD historical prices. Coins ... All Coins Compare Coins Converter Global Chart Exchanges Crypto Exchanges Decentralized Exchanges Derivatives NFT NFT ... 2024-10-08 $1,231,091,696,620 $33,

In [17]:
# Updated agent_node for Researcher (for testing):
def agent_node(state, agent, name):
    result = agent.invoke(state)
    print(f"Debug: Agent {name} invoked with state: {state}")
    print(f"Debug: Result from agent {name}: {result}")

    # If the agent is the researcher, extract research data
    if name == "Researcher":
        if isinstance(result, str) and "Error" in result:
            print("Debug: Error occurred in Research Agent. Retrying...")
        else:
            print(f"Debug: Extracted Research Data: {result}")

    # Filter Python code if invoking the chart generator
    if name == "chart_generator" or name == "code_generator":
        result = filter_python_code(result)

    # Handle different result types
    if hasattr(result, 'dict'):
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    else:
        result = AIMessage(content=str(result), name=name)

    return {
        "messages": [result],
        "sender": name,
    }

# Usage example:
# Test the Researcher agent again with the updated extraction logic
test_agent('Researcher', research_agent, "What is the Bitcoin price over the past 5 days?")


Step 1 - Starting Tool Evaluation
Step 1.1 self.bound_tools
type(self.bound_tools) <class 'list'>
self.bound_tools [TavilySearchResults(max_results=4)]
Debug: Starting tool evaluation for input: System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to answer the question. If you can't answer fully, another assistant will help. Prefix final responses with FINAL ANSWER.
Tools available: tavily_search_results_json.
Extract the data in a list such can can be used for the chart generator.
Human: What is the Bitcoin price over the past 5 days?
tool_instances: [TavilySearchResults(max_results=4)]
Debug: Result from tool tavily_search_results_json: [{'url': 'https://coinmarketcap.com/currencies/bitcoin/historical-data/', 'content': 'Cryptocurrencies. Bitcoin. Historical Data. The live Bitcoin price today is $63,028.60 USD with a 24-hour trading volume of $24,033,132,588.16 USD. We update our BTC to USD price in real-time.'}, {'url': 'https://coinma

In [18]:
# Test the code generator agent
test_agent('code_generator', code_generator_agent, "Generate Python code to plot the Bitcoin price.")


Step 1 - Starting Tool Evaluation
prompts ["System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to answer the question. If you can't answer fully, another assistant will help. Prefix final responses with FINAL ANSWER.\nTools available: .\nGenerate Python code using matplotlib for drawing the chart based on the researcher's data.\nHuman: Generate Python code to plot the Bitcoin price."]
Debug: Agent code_generator invoked with state: {'messages': [HumanMessage(content='Generate Python code to plot the Bitcoin price.')], 'sender': 'code_generator'}
Debug: Result from agent code_generator: finance.pyplot as plt plt.ion() plt.title("Bitcoin Price") plt.xlabel("Date") plt.ylabel("Price") plt.date() plt.plot(x, y, marker=".", color= "#98c9f5") plt.savefig("Bitcoin_Price.png")
Agent Name: code_generator
Test Input: Generate Python code to plot the Bitcoin price.
Agent Output: No Python code blocks found.

----------------------------------------

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, StateGraph
import functools
import re
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
import operator
from typing import Annotated, Sequence, TypedDict

# Define the object passed between nodes
class AgentState(TypedDict):
    messages: Annotated[Sequence[AIMessage], operator.add]
    sender: str

# Tool Initialization
tavily_tool = TavilySearchResults(max_results=4)
repl = PythonREPL()

# Define Python REPL Tool
@tool
def python_repl(code: Annotated[str, "The python code to execute to generate your chart."]):
    """Use this to execute python code. If you want to see the output of a value, you should print it out with `print(...)`. This is visible to the user."""
    try:
        exec(code)
        result = ""
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."

# Function to handle research data (Researcher Agent Node)
def agent_node(state, agent, name):
    result = agent.invoke(state)
    print(f"Debug: Agent {name} invoked with state: {state}")
    print(f"Debug: Result from agent {name}: {result}")

    # If the agent is the researcher, print the research data
    if name == "Researcher":
        if isinstance(result, str) and "Error" in result:
            print("Debug: Error occurred in Research Agent. Retrying...")
        else:
            # Print the full result directly as research data
            print(f"Debug: Extracted Research Data: {result}")

    # Filter Python code if invoking the chart generator
    if name == "chart_generator" or name == "code_generator":
        result = filter_python_code(result)

    # Handle different result types
    if hasattr(result, 'dict'):
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    else:
        result = AIMessage(content=str(result), name=name)

    return {
        "messages": [result],
        "sender": name,
    }

# Simplified agent creation function
def create_agent(llm, tools, system_message: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful AI assistant, collaborating with other assistants. "
             "Use the provided tools to answer the question. If you can't answer fully, another assistant will help. "
             "Prefix final responses with FINAL ANSWER.\nTools available: {tool_names}.\n{system_message}"),
            MessagesPlaceholder(variable_name="messages"),
        ]
    ).partial(system_message=system_message, tool_names=", ".join([tool.name for tool in tools]))

    return prompt | llm.bind_tools(
        tools=tools,
        model_id="ibm/granite-13b-instruct-v2",
        url=ibm_params.url,
        apikey=ibm_params.api_key,
        project_id=ibm_params.project_id,
        params=parameters
    )

# Extract Python code from the result
def filter_python_code(result):
    """
    Extract all valid Python code blocks from the AI-generated response.
    Looks for code blocks enclosed within triple backticks (```python ... ```).
    Returns a list of code blocks, or a message if none are found.
    """
    code_blocks = re.findall(r"```python\s*(.*?)```", str(result), re.DOTALL)
    if code_blocks:
        return [code.strip() for code in code_blocks]
    else:
        return "No Python code blocks found."

# Function to invoke agents and handle their results
def agent_node(state, agent, name):
    result = agent.invoke(state)
    print(f"Debug: Agent {name} invoked with state: {state}")
    print(f"Debug: Result from agent {name}: {result}")

    # If the agent is the researcher, extract research data
    if name == "Researcher":
        if isinstance(result, str) and "Error" in result:
            print("Debug: Error occurred in Research Agent. Retrying...")
        else:
            # Print the result directly
            print(f"Debug: Extracted Research Data: {result}")

    # Filter Python code if invoking the chart generator
    if name == "chart_generator" or name == "code_generator":
        result = filter_python_code(result)

    # Handle different result types
    if hasattr(result, 'dict'):
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    else:
        result = AIMessage(content=str(result), name=name)

    return {
        "messages": [result],
        "sender": name,
    }

# Define agents and their nodes
research_agent = create_agent(
    WatsonxLLM, [tavily_tool], system_message="Provide accurate data for the chart generator."
)
chart_agent = create_agent(
    WatsonxLLM, [python_repl], system_message="Any charts you display will be visible to the user."
)

# Python Code Generator Agent (new)
code_generator_agent = create_agent(
    WatsonxLLM, [], system_message="Generate Python code using matplotlib for drawing the chart based on the researcher's data."
)

research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")
chart_node = functools.partial(agent_node, agent=chart_agent, name="chart_generator")
code_generator_node = functools.partial(agent_node, agent=code_generator_agent, name="code_generator")

# Create tool node
tools = [tavily_tool, python_repl]
tool_node = ToolNode(tools)

# Define router for graph transitions
def router(state) -> str:
    last_message = state["messages"][-1]
    print(f"Debug: Last message in state: {last_message}")
    
    if last_message.tool_calls:
        print("Debug: Routing to call_tool")
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        print("Debug: Routing to __end__")
        return "__end__"
    print("Debug: Continuing to next node")
    return "continue"

# Define the graph and its nodes
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("chart_generator", chart_node)
workflow.add_node("code_generator", code_generator_node)
workflow.add_node("call_tool", tool_node)
workflow.add_conditional_edges("Researcher", router, {"continue": "code_generator", "call_tool": "call_tool", "__end__": END})
workflow.add_conditional_edges("code_generator", router, {"continue": "chart_generator", "call_tool": "call_tool", "__end__": END})
workflow.add_conditional_edges("chart_generator", router, {"continue": "Researcher", "call_tool": "call_tool", "__end__": END})
workflow.add_conditional_edges("call_tool", lambda x: x["sender"], {"Researcher": "Researcher", "chart_generator": "chart_generator", "code_generator": "code_generator"})

# Set the entry point of the workflow
workflow.set_entry_point("Researcher")

# Compile the workflow for execution
graph = workflow.compile()

# Execute the graph stream with a test message
events = graph.stream(
    {
        "messages": [
            HumanMessage(content="Fetch the bitcoin price over the past 5 days, then draw a line graph of it.")
        ]
    },
    {"recursion_limit": 50}
)

# Output the results
for s in events:
    print(s)
    print("----")


Step 1 - Starting Tool Evaluation
Step 1.1 self.bound_tools
type(self.bound_tools) <class 'list'>
self.bound_tools [TavilySearchResults(max_results=4)]
Debug: Starting tool evaluation for input: System: You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to answer the question. If you can't answer fully, another assistant will help. Prefix final responses with FINAL ANSWER.
Tools available: tavily_search_results_json.
Provide accurate data for the chart generator.
Human: Fetch the bitcoin price over the past 5 days, then draw a line graph of it.
tool_instances: [TavilySearchResults(max_results=4)]
Debug: Result from tool tavily_search_results_json: [{'url': 'https://www.coingecko.com/en/coins/bitcoin/historical_data', 'content': 'Get Bitcoin (BTC) USD historical prices. Coins: 14,755. Exchanges: 1,205. Market Cap: ... All Coins Compare Coins Converter Global Chart Exchanges Crypto Exchanges Decentralized Exchanges Derivatives NFT NFT ... 2024-09-

KeyboardInterrupt: 

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, StateGraph
import functools
import re
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
import operator
from typing import Annotated, Sequence, Literal, TypedDict

# %% [markdown]
# ## Create Agents

# Create agent function based on provided structure
def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

# %% [markdown]
# ## Define Tools

# Tavily Search Tool
tavily_tool = TavilySearchResults(max_results=5)

# Python REPL for code execution
repl = PythonREPL()

@tool
def python_repl(
    code: Annotated[str, "The python code to execute to generate your chart."]
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

# %% [markdown]
# ## Define State

# This defines the object that is passed between each node in the graph.
class AgentState(TypedDict):
    messages: Annotated[Sequence[AIMessage], operator.add]
    sender: str

# %% [markdown]
# ## Define Agent Nodes

# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # Convert the agent output into a format suitable to append to the global state
    if isinstance(result, AIMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        "sender": name,  # Track the sender so we know who to pass to next
    }

# Initialize LLM
llm = ChatOpenAI(model="gpt-4-1106-preview")

# Create the Research agent and node
research_agent = create_agent(
    llm,
    [tavily_tool],
    system_message="You should provide accurate data for the chart_generator to use.",
)
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# Create the Chart Generator agent and node
chart_agent = create_agent(
    llm,
    [python_repl],
    system_message="Any charts you display will be visible by the user.",
)
chart_node = functools.partial(agent_node, agent=chart_agent, name="chart_generator")

# %% [markdown]
# ## Define Tool Node

# Create the tool node to manage tools
tools = [tavily_tool, python_repl]
tool_node = ToolNode(tools)

# %% [markdown]
# ## Define Edge Logic

# Routing logic to manage agent transitions
def router(state) -> Literal["call_tool", "__end__", "continue"]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        return "__end__"
    return "continue"

# %% [markdown]
# ## Define the Graph

# Create the workflow for the agents
workflow = StateGraph(AgentState)

workflow.add_node("Researcher", research_node)
workflow.add_node("chart_generator", chart_node)
workflow.add_node("call_tool", tool_node)

# Define conditional edges for transitioning between agents
workflow.add_conditional_edges(
    "Researcher",
    router,
    {"continue": "chart_generator", "call_tool": "call_tool", "__end__": END},
)
workflow.add_conditional_edges(
    "chart_generator",
    router,
    {"continue": "Researcher", "call_tool": "call_tool", "__end__": END},
)

workflow.add_conditional_edges(
    "call_tool",
    lambda x: x["sender"],  # Route back to the agent who invoked the tool
    {
        "Researcher": "Researcher",
        "chart_generator": "chart_generator",
    },
)

# Set the entry point of the workflow
workflow.set_entry_point("Researcher")

# Compile the graph for execution
graph = workflow.compile()

# %% [markdown]
# ## Execute the Graph

# Helper function to execute the graph and print the result events
def execute_graph(content: str):
    print(f"Executing graph with content: {content}")
    events = graph.stream(
        {
            "messages": [HumanMessage(content=content)],
        },
        {"recursion_limit": 150}  # Max steps in the graph
    )
    # Output the events from the execution
    for event in events:
        print(event)
        print("----")

# Execute the graph with an example request
execute_graph("Fetch the Bitcoin price over the past 5 days, then draw a line graph of it.")



Executing graph with content: Fetch the Bitcoin price over the past 5 days, then draw a line graph of it.
{'Researcher': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7dISAQQmyUJ7YK1vXgr3eXfa', 'function': {'arguments': '{"query":"Bitcoin price over the past 5 days"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 212, 'total_tokens': 237, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4ffd1a7e-c52b-43e8-9d05-cc528e409df7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Bitcoin price over the past 5 days'}, 'id': 'call_7dISAQQmyUJ7YK1vXgr3eXfa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 212, 'output_tokens': 25, 'total_tokens': 237})], 'sender': '

Python REPL can execute arbitrary code. Use with caution.


{'chart_generator': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RmAtmgR36OeKxiledIDipUjF', 'function': {'arguments': '{"code":"import yfinance as yf\\nimport pandas as pd\\nimport matplotlib.pyplot as plt\\n\\n# Fetching the Bitcoin price data for the past 5 days\\nbtc = yf.Ticker(\'BTC-USD\')\\nbtc_df = btc.history(period=\'5d\')\\n\\n# Displaying the dataframe to ensure we have the data\\nprint(btc_df)\\n\\n# Plotting the closing price\\nbtc_df[\'Close\'].plot(title=\'Bitcoin Price Over the Past 5 Days\', figsize=(10, 5))\\nplt.xlabel(\'Date\')\\nplt.ylabel(\'Price (USD)\')\\nplt.grid(True)\\nplt.show()"}', 'name': 'python_repl'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 1166, 'total_tokens': 1316, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'tool_ca

KeyboardInterrupt: 